adapted code from video : https://www.youtube.com/watch?v=Q1i4bIIFOFc

In [ ]:
!nvidia-smi

Sun Mar 24 18:45:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%capture
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets

In [ ]:
import pandas as pd
import os
from google.colab import drive
from datasets import load_dataset

drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [ ]:
train_data_path = '/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/data/pii-detection-removal-from-educational-data/train.json'
test_data_path = '/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/data/pii-detection-removal-from-educational-data/test.json'

In [ ]:
data = load_dataset('json', data_files=train_data_path, split='train')
data

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['trailing_whitespace', 'tokens', 'document', 'labels', 'full_text'],
    num_rows: 6807
})

In [ ]:
data.features
# the labels are a Sequence of Value classes
# need to turn Value into ClassLabel

{'trailing_whitespace': Sequence(feature=Value(dtype='bool', id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'document': Value(dtype='int64', id=None),
 'labels': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'full_text': Value(dtype='string', id=None)}

In [ ]:
pd.DataFrame(data[:])[['tokens', 'labels']].iloc[0]

tokens    [Design, Thinking, for, innovation, reflexion,...
labels    [O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-...
Name: 0, dtype: object

In [ ]:
example = data[0]
pd.DataFrame([example["tokens"][:20], example["labels"][:20]],  # only visualise the 20 first tokens
['Tokens', 'Labels'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,Design,Thinking,for,innovation,reflexion,-,Avril,2021,-,Nathalie,Sylla,\n\n,Challenge,&,selection,\n\n,The,tool,I,use
Labels,O,O,O,O,O,O,O,O,O,B-NAME_STUDENT,I-NAME_STUDENT,O,O,O,O,O,O,O,O,O


In [ ]:
import datasets


data = data.cast_column("labels", datasets.Sequence(datasets.ClassLabel(names=['O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'B-URL_PERSONAL',
 'B-ID_NUM',
 'B-EMAIL',
 'I-STREET_ADDRESS',
 'I-PHONE_NUM',
 'B-USERNAME',
 'B-PHONE_NUM',
 'B-STREET_ADDRESS',
 'I-URL_PERSONAL',
 'I-ID_NUM'])))
data.features

Casting the dataset:   0%|          | 0/6807 [00:00<?, ? examples/s]

{'trailing_whitespace': Sequence(feature=Value(dtype='bool', id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'document': Value(dtype='int64', id=None),
 'labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None), length=-1, id=None),
 'full_text': Value(dtype='string', id=None)}

In [ ]:
pd.DataFrame(data[:])[['tokens', 'labels']].iloc[0]

tokens    [Design, Thinking, for, innovation, reflexion,...
labels    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, ...
Name: 0, dtype: object

In [ ]:
example = data[0]
pd.DataFrame([example["tokens"][:20], example["labels"][:20]],  # only visualise the 20 first tokens
['Tokens', 'Labels'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,Design,Thinking,for,innovation,reflexion,-,Avril,2021,-,Nathalie,Sylla,\n\n,Challenge,&,selection,\n\n,The,tool,I,use
Labels,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0


In [ ]:
# remove irrelevant features
data = data.remove_columns(["full_text", "trailing_whitespace", "document"])
data

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 6807
})

In [ ]:
path_fake = '/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/data/fake_data/concatenated_kaggle_data.json'
fake_data = load_dataset('json', data_files=path_fake)
fake_data

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'tokens', 'index'],
        num_rows: 6082
    })
})

In [ ]:
fake_data = fake_data.remove_columns("index")
fake_data

DatasetDict({
    train: Dataset({
        features: ['labels', 'tokens'],
        num_rows: 6082
    })
})

In [ ]:
fake_data['train'].features

{'labels': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [ ]:
fake_data['train'] = fake_data['train'].cast_column("labels", datasets.Sequence(datasets.ClassLabel(names=['O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'B-URL_PERSONAL',
 'B-ID_NUM',
 'B-EMAIL',
 'I-STREET_ADDRESS',
 'I-PHONE_NUM',
 'B-USERNAME',
 'B-PHONE_NUM',
 'B-STREET_ADDRESS',
 'I-URL_PERSONAL',
 'I-ID_NUM'])))
fake_data['train'].features

Casting the dataset:   0%|          | 0/6082 [00:00<?, ? examples/s]

{'labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [ ]:
from datasets import concatenate_datasets
# assert data.features.type == fake_data['train'].features.type
big_dataset = concatenate_datasets([data, fake_data['train']])
big_dataset

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 12889
})

In [ ]:
big_dataset.features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None), length=-1, id=None)}

In [ ]:
example = big_dataset[8000]
pd.DataFrame([example["tokens"][:20], example["labels"][:20]],  # only visualise the 20 first tokens
['Tokens', 'Labels'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,I,am,Emiko,"Gauthier,",a,highly,driven,and,results-oriented,professional,with,a,passion,for,driving,innovation,and,delivering,exceptional,results.
Labels,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
big_dataset = big_dataset.rename_column("labels", "ner_tags")
big_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 12889
})

### split train-validation (test)

In [ ]:
data = big_dataset.train_test_split(test_size=0.2)
data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 10311
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2578
    })
})

In [ ]:
tags = data['train'].features['ner_tags'].feature
tags.names

['O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'B-URL_PERSONAL',
 'B-ID_NUM',
 'B-EMAIL',
 'I-STREET_ADDRESS',
 'I-PHONE_NUM',
 'B-USERNAME',
 'B-PHONE_NUM',
 'B-STREET_ADDRESS',
 'I-URL_PERSONAL',
 'I-ID_NUM']

In [ ]:
tags_dict = {tag: 0 for tag in tags.names}
print(tags_dict)

def count_tags(example):
    for tag in example['ner_tags']:
        tag = tags.int2str(tag)
        tags_dict[tag] += 1

# count tags in the train set
data['train'].map(count_tags)
print('Train dataset tags:', tags_dict)

# reset dict and count the tags in the test set
tags_dict = {tag: 0 for tag in tags.names}
data['test'].map(count_tags)
print('Validation dataset tags:', tags_dict)

# data is unbalanced - 'O' label has overwhlemingly more examples
# sutend name label is relatively balanced

{'O': 0, 'B-NAME_STUDENT': 0, 'I-NAME_STUDENT': 0, 'B-URL_PERSONAL': 0, 'B-ID_NUM': 0, 'B-EMAIL': 0, 'I-STREET_ADDRESS': 0, 'I-PHONE_NUM': 0, 'B-USERNAME': 0, 'B-PHONE_NUM': 0, 'B-STREET_ADDRESS': 0, 'I-URL_PERSONAL': 0, 'I-ID_NUM': 0}


Map:   0%|          | 0/10311 [00:00<?, ? examples/s]

Train dataset tags: {'O': 5963327, 'B-NAME_STUDENT': 12457, 'I-NAME_STUDENT': 6452, 'B-URL_PERSONAL': 999, 'B-ID_NUM': 173, 'B-EMAIL': 3669, 'I-STREET_ADDRESS': 8286, 'I-PHONE_NUM': 3470, 'B-USERNAME': 1101, 'B-PHONE_NUM': 2325, 'B-STREET_ADDRESS': 3006, 'I-URL_PERSONAL': 1, 'I-ID_NUM': 1}


Map:   0%|          | 0/2578 [00:00<?, ? examples/s]

Validation dataset tags: {'O': 1508187, 'B-NAME_STUDENT': 3252, 'I-NAME_STUDENT': 1621, 'B-URL_PERSONAL': 265, 'B-ID_NUM': 35, 'B-EMAIL': 918, 'I-STREET_ADDRESS': 2046, 'I-PHONE_NUM': 848, 'B-USERNAME': 290, 'B-PHONE_NUM': 594, 'B-STREET_ADDRESS': 788, 'I-URL_PERSONAL': 0, 'I-ID_NUM': 0}


In [ ]:
pd.DataFrame(data['train'][:])[['tokens', 'ner_tags']].iloc[0]

tokens      [ , Title, :, Mind, Mapping, :, A, Creative, a...
ner_tags    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: 0, dtype: object

In [ ]:
tags = data['train'].features['ner_tags'].feature

index2tag = {idx:tag for idx, tag in enumerate(tags.names)}
tag2index = {tag:idx for idx, tag in enumerate(tags.names)}
index2tag, tag2index

({0: 'O',
  1: 'B-NAME_STUDENT',
  2: 'I-NAME_STUDENT',
  3: 'B-URL_PERSONAL',
  4: 'B-ID_NUM',
  5: 'B-EMAIL',
  6: 'I-STREET_ADDRESS',
  7: 'I-PHONE_NUM',
  8: 'B-USERNAME',
  9: 'B-PHONE_NUM',
  10: 'B-STREET_ADDRESS',
  11: 'I-URL_PERSONAL',
  12: 'I-ID_NUM'},
 {'O': 0,
  'B-NAME_STUDENT': 1,
  'I-NAME_STUDENT': 2,
  'B-URL_PERSONAL': 3,
  'B-ID_NUM': 4,
  'B-EMAIL': 5,
  'I-STREET_ADDRESS': 6,
  'I-PHONE_NUM': 7,
  'B-USERNAME': 8,
  'B-PHONE_NUM': 9,
  'B-STREET_ADDRESS': 10,
  'I-URL_PERSONAL': 11,
  'I-ID_NUM': 12})

In [ ]:
tags.int2str(1)

'B-NAME_STUDENT'

In [ ]:
def create_tag_names(batch):
    tag_name = {'ner_tags_str': [tags.int2str(idx) for idx in batch['ner_tags']]}
    return tag_name

In [ ]:
data = data.map(create_tag_names)
data

Map:   0%|          | 0/10311 [00:00<?, ? examples/s]

Map:   0%|          | 0/2578 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'ner_tags_str'],
        num_rows: 10311
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'ner_tags_str'],
        num_rows: 2578
    })
})

In [ ]:
pd.DataFrame(data['train'][:])[['tokens', 'ner_tags', 'ner_tags_str']].iloc[0]

tokens          [ , Title, :, Mind, Mapping, :, A, Creative, a...
ner_tags        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
ner_tags_str    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
Name: 0, dtype: object

### model building - tokenize

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = 'allenai/longformer-base-4096'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
tokenizer.is_fast

True

In [ ]:
inputs = data['train'][0]['tokens']  # these are 'pretokenized' so it needs to tokenize it further
inputs = tokenizer(inputs, is_split_into_words=True)  # we need to tell the tokenizer the data is split into words otherwize it will just consider it as one input
print(inputs.tokens())

['<s>', 'Ġ', 'ĠTitle', 'Ġ:', 'ĠMind', 'ĠM', 'apping', 'Ġ:', 'ĠA', 'ĠCreative', 'Ġand', 'ĠEffective', 'ĠTool', 'Ġfor', 'ĠSol', 'ving', 'ĠComplex', 'ĠChallenges', 'Ġ:', 'ĠA', 'ĠRef', 'lection', 'Ġby', 'ĠJose', 'ĠManuel', 'Ġ', 'ĊĊ', 'ĠIntroduction', 'Ġ:', 'Ġ', 'Ċ', 'ĠI', 'Ġ,', 'ĠJose', 'ĠManuel', 'Ġ,', 'Ġam', 'Ġa', 'Ġstudent', 'Ġcurrently', 'Ġtaking', 'Ġan', 'Ġonline', 'Ġdesign', 'Ġthinking', 'Ġcourse', 'Ġ.', 'ĠFor', 'Ġmy', 'Ġfinal', 'Ġessay', 'Ġ,', 'ĠI', 'Ġhave', 'Ġchosen', 'Ġto', 'Ġreflect', 'Ġon', 'Ġthe', 'ĠMind', 'ĠM', 'apping', 'Ġtool', 'Ġ,', 'Ġa', 'Ġcreative', 'Ġand', 'Ġeffective', 'Ġway', 'Ġto', 'Ġgenerate', 'Ġ,', 'Ġorganize', 'Ġ,', 'Ġand', 'Ġvisualize', 'Ġideas', 'Ġ.', 'ĠIn', 'Ġthis', 'Ġessay', 'Ġ,', 'ĠI', 'Ġwill', 'Ġdescribe', 'Ġmy', 'Ġchallenge', 'Ġ,', 'Ġexplain', 'Ġwhy', 'ĠI', 'Ġselected', 'Ġthe', 'ĠMind', 'ĠM', 'apping', 'Ġtool', 'Ġ,', 'Ġdiscuss', 'Ġhow', 'ĠI', 'Ġapplied', 'Ġit', 'Ġto', 'Ġmy', 'Ġchallenge', 'Ġ,', 'Ġshare', 'Ġthe', 'Ġinsights', 'ĠI', 'Ġgained', 'Ġ,', 'Ġand', 'Ġ

In [ ]:
inputs

{'input_ids': [0, 1437, 13497, 4832, 9201, 256, 12040, 4832, 83, 14424, 8, 33355, 25251, 13, 4856, 6645, 14219, 30380, 4832, 83, 8526, 20576, 30, 3071, 11294, 1437, 50140, 24474, 4832, 1437, 50118, 38, 2156, 3071, 11294, 2156, 524, 10, 1294, 855, 602, 41, 804, 1521, 2053, 768, 479, 286, 127, 507, 14700, 2156, 38, 33, 4986, 7, 4227, 15, 5, 9201, 256, 12040, 3944, 2156, 10, 3904, 8, 2375, 169, 7, 5368, 2156, 14192, 2156, 8, 39086, 2956, 479, 96, 42, 14700, 2156, 38, 40, 6190, 127, 1539, 2156, 3922, 596, 38, 3919, 5, 9201, 256, 12040, 3944, 2156, 2268, 141, 38, 5049, 24, 7, 127, 1539, 2156, 458, 5, 8418, 38, 3491, 2156, 8, 3608, 99, 38, 429, 109, 8225, 220, 86, 479, 1437, 50140, 10045, 4832, 1437, 50118, 1308, 1539, 16, 7, 1521, 10, 5068, 8, 4555, 2004, 2472, 13, 614, 111, 1425, 1232, 11, 4879, 911, 479, 152, 1539, 16, 1605, 4249, 25, 4879, 9883, 535, 7, 1733, 2156, 8, 5, 240, 13, 4555, 2004, 3374, 3150, 10275, 479, 20, 13879, 9, 42, 1539, 5738, 11, 18442, 21254, 2156, 11128, 2156, 8, 321

In [ ]:
inputs.word_ids() # 7,7 blow means thay the tokens belong to one single token/word in the original dataset

[None,
 0,
 1,
 2,
 3,
 4,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 12,
 13,
 14,
 15,
 16,
 17,
 17,
 18,
 19,
 20,
 21,
 21,
 22,
 23,
 24,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 115,
 116,
 117,
 118,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 174,
 

In [ ]:
len(inputs.tokens()), len(data['train'][0]['tokens'])

(841, 769)

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)

        elif word_id is None:
            new_labels.append(-100)  # used in Pytorch to tell it to ignore the tag

        else:
            label = labels[word_id]
            if label%2 == 1:
                label = label + 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = data['train'][0]['ner_tags']
word_ids = inputs.word_ids()
print(labels)
print(word_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
align_labels_with_tokens(labels, word_ids)

[-100,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

    all_labels = examples['ner_tags']

    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
    tokenized_inputs['labels'] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = data.map(tokenize_and_align_labels, batched=True, remove_columns=data['train'].column_names)
tokenized_dataset

Map:   0%|          | 0/10311 [00:00<?, ? examples/s]

Map:   0%|          | 0/2578 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10311
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2578
    })
})

### data collation
transform into tensors for use with pytorch/transformers lib


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_dataset['train'][i] for i in range(2)])
batch

{'input_ids': tensor([[    0,  1437, 13497,  ...,  1417,  4882,     2],
        [    0, 10045,  1437,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[-100,    0,    0,  ...,    4,    4, -100],
        [-100,    0,    0,  ..., -100, -100, -100]])}

### metrics

In [ ]:
%%capture
!pip install seqeval
!pip install evaluate

In [ ]:
import evaluate

metric = evaluate.load('seqeval')

In [ ]:
ner_fetature = data['train'].features['ner_tags']
ner_fetature

Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None), length=-1, id=None)

In [ ]:
label_names = ner_fetature.feature.names
label_names

['O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'B-URL_PERSONAL',
 'B-ID_NUM',
 'B-EMAIL',
 'I-STREET_ADDRESS',
 'I-PHONE_NUM',
 'B-USERNAME',
 'B-PHONE_NUM',
 'B-STREET_ADDRESS',
 'I-URL_PERSONAL',
 'I-ID_NUM']

In [ ]:
labels = data['train'][0]['ner_tags']
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',


In [ ]:
preditctions = labels.copy()
preditctions[2] = 'O'

metric.compute(predictions=[preditctions], references=[labels])

{'NAME_STUDENT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 4},
 'STREET_ADDRESS': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'URL_PERSONAL': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
import numpy as np

def compute_metrics(eval_preds):
  logits, labels = eval_preds

  predictions = np.argmax(logits, axis=-1)

  true_labels = [[label_names[l] for l in label if l!=-100] for label in labels]

  true_predictions = [[label_names[p] for p,l in zip(prediction, label) if l!=-100]
                      for prediction, label in zip(predictions, labels)]

  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

  return {"precision": all_metrics['overall_precision'],
          "recall": all_metrics['overall_recall'],
          "f1": all_metrics['overall_f1'],
          "accuracy": all_metrics['overall_accuracy']}

### train model

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint,
                                                       num_labels=tags.num_classes,
                                                       gradient_checkpointing=False,
                                                       id2label=index2tag,
                                                       label2id=tag2index
                                                        )

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForTokenClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config

LongformerConfig {
  "_name_or_path": "allenai/longformer-base-4096",
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-NAME_STUDENT",
    "2": "I-NAME_STUDENT",
    "3": "B-URL_PERSONAL",
    "4": "B-ID_NUM",
    "5": "B-EMAIL",
    "6": "I-STREET_ADDRESS",
    "7": "I-PHONE_NUM",
    "8": "B-USERNAME",
    "9": "B-PHONE_NUM",
    "10": "B-STREET_ADDRESS",
    "11": "I-URL_PERSONAL",
    "12": "I-ID_NUM"
  },
  "ignore_attention_mask": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-EMAIL": 5,
    "B-ID_NUM": 4,
    "B-NAME_STUDENT": 1,
    "B-PHONE_NUM": 9,
    "B-STREET_ADDRESS": 10,
    "B-URL_P

In [ ]:
model.config.num_labels

13

In [ ]:
from transformers import TrainingArguments
from tqdm import tqdm


num_epochs = 3
batch_size = 1
logging_steps = len(tokenized_dataset['train']) // batch_size

args = TrainingArguments(output_dir='/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/models/longformer_augm_data/outputs',
                         evaluation_strategy='epoch',
                         log_level="error",
                         num_train_epochs=num_epochs,
                         per_device_eval_batch_size=batch_size,
                         per_device_train_batch_size=batch_size,
                         save_steps=1e6,
                         disable_tqdm=False,
                         learning_rate=2e-5,
                         weight_decay=0.01,
                         logging_steps=logging_steps)


In [ ]:
%%time
from transformers import Trainer

trainer = Trainer(model=model,
                  args=args,
                  train_dataset=tokenized_dataset['train'],
                  eval_dataset=tokenized_dataset['test'],
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.011100,0.005844,0.970243,0.950128,0.960080,0.999122
2,0.003800,0.004797,0.976897,0.955327,0.965992,0.999275
3,0.002400,0.004408,0.972177,0.962131,0.967128,0.999330


CPU times: user 1h 18min 39s, sys: 4min 20s, total: 1h 22min 59s
Wall time: 1h 22min 36s


TrainOutput(global_step=30933, training_loss=0.005775037054000784, metrics={'train_runtime': 4955.0318, 'train_samples_per_second': 6.243, 'train_steps_per_second': 6.243, 'total_flos': 1.2873359052299988e+16, 'train_loss': 0.005775037054000784, 'epoch': 3.0})

In [ ]:
from transformers import pipeline

model_checkpoint = ''